# Model Pruning and Training Notebook

## Overview
This notebook is designed for performing model pruning and training of proposed deep learning architectures, as well as evaluating their pruned versions. It also includes testing the model architectures on a reconstructed dataset and performing metrics comparison.

## Objectives
- **Model Pruning:** Reduce Model Size as per proposed approach
- **Model Training:** Train proposed  deep learning architectures, and evaluate their effectiveness.
- **Reconstructed Dataset Testing:** Test the trained models on a reconstructed dataset to assess generalization capabilities.
- **Metrics Comparison:** Compare various metrics such as training accuracy (90%), precision (80%), validation accuracy (75% for Lenet-5 architecture), and ensure testing accuracy meets a threshold of 70% with up to 2% error acceptance.

## Specifications
- **Training Accuracy:** Targeting 90% accuracy during training.
- **Precision and Accuracy:** Aim for 80% precision and accuracy metrics.
- **Validation Accuracy:** Lenet-5 architecture is expected to achieve 75% accuracy during validation.
- **Testing Criteria:** Ensure testing accuracy meets a minimum threshold of 98%, with up to 2% error acceptance.

## Usage
This notebook serves as a comprehensive tool for deep learning model development and evaluation. It includes:
- Data preprocessing steps.
- Model architecture design and implementation.
- Hyperparameter tuning.
- Model pruning techniques.
- Training, validation, and testing phases.
- Evaluation of metrics and performance analysis.

By following the structured workflow in this notebook, users can effectively experiment with different deep learning architectures, optimize model performance through pruning, and benchmark against specified performance metrics.



In [ ]:
# importing the libraries
# import the necessary packages and attributes to be used
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from glob import glob
from PIL import Image
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras import Sequential
from tensorflow.keras import Model
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.layers import (
    Dense,
    Conv2D,
    MaxPooling2D,
    Flatten,
    BatchNormalization,
    Dropout,
    concatenate,
    ZeroPadding2D
)

### Read in metadata and Preprocessing Data

In [ ]:
# Reading in the dataset for original / reconstructed dataset as per choice
csv_path = "/kaggle/input/metadata-augmented/HAM10000_metadata_350_augmented.csv"
dataset = pd.read_csv(csv_path)
dataset.shape

In [ ]:
# Define the base directory where images are stored
base_skin_dir = os.path.join('..', 'input', '350-components-aug-img')

# Create a dictionary where the key is the image_id (without extension) and the value is the full path to the image
imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x
                     for x in glob(os.path.join(base_skin_dir, '*', '*.[jJpP][pPnNeEgG]*'))}

# Update the dataset with the corresponding image paths
# Apply a lambda function to the 'image_id' column to map the image_id to its path using the dictionary
# Replace '.png' extension from 'image_id' with an empty string before lookup, this is optional and is performed as the image id in augmented dataset for original and reconstruction contains .png extension in image id
dataset['path'] = dataset['image_id'].apply(lambda x: imageid_path_dict.get(x.replace('.png', ''), ''))

# Display the count of unique values in the 'path' column to ensure paths are correctly assigned
dataset['path'].value_counts()

In [ ]:
# List of allowed values for the 'dx' column
allowed_values = ['nc', 'mel', 'nv']

# To keep only Melanoma, Non Cancerous, Non-Meanoma, and Melocytic Nevi as classes
# Update the 'dx' column:
# Apply a lambda function to each element in the 'dx' column
# If the value is in the allowed_values list, keep it as is; otherwise, replace it with 'others'
dataset['dx'] = dataset['dx'].apply(lambda x: x if x in allowed_values else 'others')

# Display the count of unique values in the 'dx' column to ensure values have been correctly updated
dx_value_counts = dataset['dx'].value_counts()

# Print the value counts for verification
print(dx_value_counts)


In [ ]:

## To Sample traininig dataset into 4400 samples, 1100 per class
## This is performed only once and not be run again

def sample_classes(df, column, n_samples=1100, random_state=None):
    """
    Sample n_samples from each class of a specified column in a pandas DataFrame.

    Parameters:
    df (pd.DataFrame): The DataFrame to sample from.
    column (str): The name of the column to group by.
    n_samples (int): The number of samples to take from each class. Default is 1000.
    random_state (int, optional): The random state for reproducibility. Default is None.

    Returns:
    pd.DataFrame: A DataFrame containing the sampled rows.
    """
    # Filter out rows where 'path' is an empty string
    df = df[df['path'] != ""]
    sampled_df = df.groupby(column).apply(lambda x: x.sample(n=n_samples, random_state=random_state))
    
    # Reset the index to get a clean DataFrame
    sampled_df = sampled_df.reset_index(drop=True)
    
    return sampled_df



augmented_df = dataset
print(f"Augmented Data Samples: {augmented_df.shape} \nDistribution: \n{augmented_df['dx'].value_counts()}")
## Save the training dataset
augmented_df.to_csv('training_origina350.csv')

In [ ]:
# Define a dictionary to map lesion type codes to their human-readable names
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma',
    'nc': 'No_Skin_Disease',
    'others': 'Others'
}

# Map the 'dx' column to human-readable names using the lesion_type_dict
# The .map method is used with the .get method of the dictionary to ensure any missing keys return None
augmented_df['cell_type'] = augmented_df['dx'].map(lesion_type_dict.get)

# Convert the human-readable lesion types to categorical codes
# pd.Categorical assigns a unique integer code to each category (lesion type) in 'cell_type'
augmented_df['cell_type_idx'] = pd.Categorical(augmented_df['cell_type']).codes

# Display the first few rows of the dataframe to verify the changes
augmented_df_head = augmented_df.head()

# Print the first few rows for verification
print(augmented_df_head)

In [ ]:
# Extract Category  mapping of the cell
category_mapping = dict(enumerate(pd.Categorical(augmented_df['cell_type']).categories))

print("Category Mapping (cell_type_idx to cell_type):")
print(category_mapping)

In [ ]:
# Replace zero values in the 'age' column with the median age
# .loc is used to select rows where 'age' is 0.0, and those values are replaced with the median age of the 'age' column
augmented_df['age'].loc[augmented_df['age'] == 0.0] = augmented_df.age.median()

# Fill any remaining NaN values in the 'age' column with 55
# .fillna is used to replace NaN values with the specified value (55)
augmented_df['age'].fillna(55, inplace=True)

# Check for any remaining missing values in the dataset
# .isna().sum() returns the count of NaN values for each column
missing_values_count = augmented_df.isna().sum()

# Print the count of missing values for verification
print(missing_values_count)

In [ ]:
# Define the size to which each image should be resized
# This size is to be changed as we change image resolution to train the models
size = (450, 600)

# Load images into the DataFrame
# The 'path' column is used to open each image file
# Each image is opened using PIL's Image.open, resized to the specified size, and converted to a numpy array
augmented_df['image'] = augmented_df['path'].map(lambda x: np.asarray(Image.open(x).resize(size)))

# Display the first few rows of the updated DataFrame to verify the changes
augmented_df_head = augmented_df.head()

# Print the first few rows for verification
print(augmented_df_head)

### Splitting the data into test and training sets

In [ ]:
# Inform the user that data processing is starting
print("[INFO] processing data...")

# Partition the data into training and testing splits
# The DataFrame 'augmented_df' is split into features (attributes) and target (images)
# 'train_test_split' is used to split the data
# The feature columns selected are 'age', 'sex', 'localization', and 'cell_type_idx'
# The target column is 'image'
# 75% of the data is used for training and the remaining 25% is used for testing
# 'test_size=0.25' specifies the proportion of the dataset to include in the test split
# 'random_state=4142' ensures reproducibility of the random splitting
train_attr, test_attr, train_img, test_img = train_test_split(
    augmented_df[['age', 'sex', 'localization', 'cell_type_idx']], 
    augmented_df['image'], 
    test_size=0.25, 
    random_state=4142
)

# Display the shapes of the resulting splits to verify the partitioning
print(f"Train attributes shape: {train_attr.shape}")
print(f"Test attributes shape: {test_attr.shape}")
print(f"Train images shape: {train_img.shape}")
print(f"Test images shape: {test_img.shape}")

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

def normalize_and_encode(df, numeric_cols, categorical_cols, scaler_path=None, encoder_path=None):
    """
    Normalize numeric columns and encode categorical columns in the DataFrame.

    Parameters:
    df (pd.DataFrame): The DataFrame to process.
    numeric_cols (list of str): List of column names for numeric features to be standardized.
    categorical_cols (list of str): List of column names for categorical features to be label encoded.
    scaler_path (str or None): Optional. File path to save StandardScaler object.
    encoder_path (str or None): Optional. File path to save LabelEncoder object.

    Returns:
    pd.DataFrame: The DataFrame with normalized numeric columns and encoded categorical columns.

    Raises:
    ValueError: If both numeric_cols and categorical_cols are empty.
    """
    
    # Check if input lists are empty
    if not numeric_cols and not categorical_cols:
        raise ValueError("Please provide at least one numeric or categorical column name.")
    
    # Standardize numeric columns using StandardScaler
    if numeric_cols:
        scaler = StandardScaler()
        df[numeric_cols] = scaler.fit_transform(df[numeric_cols])
        # Save scaler object if path is provided
        if scaler_path:
            with open(scaler_path, 'wb') as f:
                pickle.dump(scaler, f)
    
    # Label encode categorical columns using LabelEncoder
    if categorical_cols:
        for col in categorical_cols:
            encoder = LabelEncoder()
            df[col] = encoder.fit_transform(df[col])
            # Save encoder object if path is provided
            if encoder_path:
                with open(encoder_path, 'wb') as f:
                    pickle.dump(encoder, f)
    
    return df

# Example usage
df = pd.DataFrame({'age': [25, 35, 45], 'sex': ['male', 'female', 'male'], 'income': [50000, 60000, 70000]})
normalized_df = normalize_and_encode(df, numeric_cols=['age', 'income'], categorical_cols=['sex'], scaler_path='scaler.pkl', encoder_path='encoder.pkl')
print(normalized_df)


In [ ]:
# Normalize and encode training and testing attribute data
train_attr = normalize_and_encode(train_attr, ['age'], ['sex','localization'])
test_attr = normalize_and_encode(test_attr, ['age'], ['sex','localization'])

# Convert target column 'cell_type_idx' to categorical (one-hot encoding) for model training
train_y = to_categorical(list(train_attr['cell_type_idx']), 4)
test_y = to_categorical(list(test_attr['cell_type_idx']), 4)

# Display shapes for verification
print(f"Train attributes shape: {train_attr.shape}")
print(f"Test attributes shape: {test_attr.shape}")
print(f"Train images shape: {train_img.shape}")
print(f"Test images shape: {test_img.shape}")
print(f"Train labels shape: {train_y.shape}")
print(f"Test labels shape: {test_y.shape}")

### Proposed architectures

Note - Refer cnn_concepts.ipynb for techincal definitions of concepts

### Lenet-5 based Neural Network

In [ ]:

def create_mixed_nn():
    """
    Creates a mixed neural network model combining MLP (text model) and CNN (image model) branches for prediction.

    Returns:
    tf.keras.Model: A TensorFlow Keras Model instance representing the mixed neural network.
    """
    # Define input shapes and number of classes
    input_shape_text = (3,)
    input_shape_image = (600, 450, 3)
    num_classes = 4

    # Text (MLP) Model
    text_model = Sequential(name='text_model')
    text_model.add(Dense(4, input_shape=input_shape_text, activation="relu"))
    text_model.add(Dropout(0.25))
    text_model.add(Dense(6, activation="relu"))
    text_model.add(Dense(2, activation="relu"))
    text_model.add(Dropout(0.25))
    text_model.add(Dense(units=num_classes, activation="softmax"))

    # Image (CNN) Model
    image_model = Sequential(name='image_model')
    image_model.add(Conv2D(6, (5, 5), activation='relu', input_shape=input_shape_image))
    image_model.add(MaxPooling2D((2, 2)))
    image_model.add(BatchNormalization())
    image_model.add(Conv2D(16, (5, 5), activation='relu'))
    image_model.add(MaxPooling2D((2, 2)))
    image_model.add(BatchNormalization())
    image_model.add(Flatten())
    image_model.add(Dense(84, activation='relu'))
    image_model.add(Dense(120, activation='relu'))

    # Concatenate Outputs
    combined_output = concatenate([text_model.output, image_model.output])

    # Final Dense layers for classification
    x = Dense(16, activation="relu")(combined_output)
    x = Dense(num_classes, activation="softmax")(x)

    # Create a model that includes both text and image inputs, and outputs the final classification
    mixed_model = tf.keras.Model([text_model.input, image_model.input], x, name='mixed_model')

    return mixed_model

# Enable eager execution for debugging (optional)
tf.config.run_functions_eagerly(True)

# Create the mixed data model
mixed_data_model = create_mixed_nn()

# Compile the model with optimizer, loss function, and metrics
mixed_data_model.compile(
    optimizer=Adam(learning_rate=0.0002),
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy', 'Precision', 'AUC'],  # Add additional metrics for evaluation
)

# Display model summary
mixed_data_model.summary()

### Alexnet based Neural Network

In [ ]:
def create_mixed_nn():
    """
    Creates a mixed neural network model combining MLP (text model) and deep CNN (image model) branches for prediction.

    Returns:
    tf.keras.Model: A TensorFlow Keras Model instance representing the mixed neural network.
    """
    # Define input shape for the image model and number of classes
    input_shape_image = (600, 450, 3)
    num_classes = 4

    # Text (MLP) Model
    text_model = Sequential(name='text_model')
    text_model.add(Dense(4, input_dim=3, activation="relu"))
    text_model.add(BatchNormalization())
    text_model.add(Dropout(0.25))
    text_model.add(Dense(6, activation="relu"))
    text_model.add(BatchNormalization())
    text_model.add(Dense(2, activation="relu"))
    text_model.add(BatchNormalization())
    text_model.add(Dropout(0.25))
    text_model.add(Dense(units=num_classes, activation="softmax"))

    # Image (CNN) Model
    model = Sequential(name='image_model')
    model.add(Conv2D(64, (5, 5), strides=(2, 2), input_shape=input_shape_image, activation='relu', padding='valid'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((3, 3), padding='valid'))

    model.add(ZeroPadding2D((2, 2)))
    model.add(Conv2D(192, (5, 5), strides=(1, 1), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((3, 3), padding='valid'))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(384, (3, 3), activation='relu', padding='same'))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(384, (3, 3), activation='relu', padding='same'))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((3, 3), padding='valid'))

    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))

    # Concatenate Outputs
    combined_output = concatenate([text_model.output, model.output])

    # Final Dense layers for classification
    x = Dense(8, activation="relu")(combined_output)
    x = Dense(num_classes, activation="softmax")(x)

    # Create a model that includes both text and image inputs, and outputs the final classification
    mixed_model = Model([text_model.input, model.input], x, name='mixed_model')

    return mixed_model

# Enable eager execution for debugging (optional)
tf.config.run_functions_eagerly(True)

# Create the mixed data model
mixed_data_model = create_mixed_nn()

# Compile the model with optimizer, loss function, and metrics
mixed_data_model.compile(
    optimizer=Adam(learning_rate=0.0002),
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy', 'Precision', 'AUC'],  # Add additional metrics for evaluation
)

# Display model summary
mixed_data_model.summary()

#### Custom Neural Network - 1

In [ ]:
def create_mixed_nn():
    """
    Creates a mixed neural network model combining text (MLP) and image (CNN) inputs.

    Returns:
    model (tf.keras.Model): Compiled Keras model.
    """
    input_shape_1 = (600, 450, 3)  # Input shape for image (CNN) model
    num_classes = 4  # Number of output classes

    # Text (MLP) Model
    text_model = Sequential(name='text_model')
    text_model.add(Dense(4, input_dim=3, activation="relu"))
    text_model.add(Dropout(0.25))
    text_model.add(Dense(6, activation="relu"))
    text_model.add(Dense(2, activation="relu"))
    text_model.add(Dropout(0.25))
    text_model.add(Dense(units=1, activation="relu"))  # Adjust units for specific classes

    # Image (CNN) Model
    image_model = Sequential(name='image_model')
    image_model.add(Conv2D(16, 7, activation="relu", input_shape=input_shape_1))
    image_model.add(MaxPooling2D())
    image_model.add(BatchNormalization())
    image_model.add(Dropout(0.25))

    image_model.add(Conv2D(48, 5, activation="relu"))
    image_model.add(MaxPooling2D())
    image_model.add(BatchNormalization())
    image_model.add(Dropout(0.1))

    image_model.add(Conv2D(92, 3, activation="relu"))
    image_model.add(MaxPooling2D())
    image_model.add(BatchNormalization())
    image_model.add(Dropout(0.1))

    image_model.add(Conv2D(192, 3, activation="relu"))
    image_model.add(MaxPooling2D())
    image_model.add(BatchNormalization())
    image_model.add(Dropout(0.1))

    image_model.add(Conv2D(192, 3, activation="relu"))
    image_model.add(MaxPooling2D())
    image_model.add(BatchNormalization())
    image_model.add(Dropout(0.1))

    image_model.add(Conv2D(256, 3, activation="relu"))
    image_model.add(MaxPooling2D())
    image_model.add(BatchNormalization())
    image_model.add(Dropout(0.1))

    image_model.add(Flatten())
    image_model.add(Dense(1024, activation="relu"))
    image_model.add(Dropout(0.15))
    image_model.add(BatchNormalization())

    image_model.add(Dense(1024, activation="relu"))
    image_model.add(Dropout(0.15))
    image_model.add(BatchNormalization())

    image_model.add(Dense(512, activation="relu"))  # Adjust units for specific classes
    image_model.add(Dropout(0.1))

    # Concatenate Text and Image Models
    combined_output = concatenate([text_model.output, image_model.output])
    # Concatenation combines the output of the text_model and image_model, preserving their features in a single vector.

    # Final Dense Layers
    x = Dense(16, activation="relu")(combined_output)
    x = Dense(num_classes, activation="softmax")(x)

    # Create Model
    model = Model([text_model.input, image_model.input], x, name='mixed_model')

    return model

# Enable eager execution for immediate execution and debugging
tf.config.run_functions_eagerly(True)

# Create and Compile the Mixed Neural Network Model
mixed_data_model = create_mixed_nn()
mixed_data_model.compile(
    optimizer=Adam(learning_rate=0.0002),
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy', 'Precision', 'AUC']
)

# Print Model Summary
mixed_data_model.summary()

### Custom Neural Network - 2

In [ ]:

def resnet_block(x, filters, kernel_size=3, strides=1):
    """
    ResNet block with optional downsampling.
    
    Args:
    x (tensor): Input tensor.
    filters (int): Number of filters for the convolutional layers.
    kernel_size (int): Size of the convolutional kernel.
    strides (int): Stride size for the convolutional layers.

    Returns:
    tensor: Output tensor after applying the ResNet block operations.
    """
    shortcut = x
    x = Conv2D(filters, kernel_size, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    
    # Shortcut connection
    if strides != 1 or shortcut.shape[-1] != filters:
        shortcut = Conv2D(filters, 1, strides=strides, padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)
    
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

def create_mixed_nn_resnet(): 
    """
    Creates a mixed neural network model combining text (MLP) and image (CNN) inputs,
    including ResNet blocks for the image model.
    
    Returns:
    model (tf.keras.Model): Compiled Keras model.
    """
    input_shape_1 = (100, 200, 3)  # Input shape for image (CNN) model
    num_classes = 8
    
    # Text (MLP) Model
    text_model = Sequential()
    text_model.add(Dense(4, input_dim=3, activation="relu"))
    text_model.add(BatchNormalization())
    text_model.add(Dropout(0.25))
    
    text_model.add(Dense(6, input_dim=3, activation="relu"))
    text_model.add(BatchNormalization())
    
    text_model.add(Dense(2, input_dim=3, activation="relu"))
    text_model.add(BatchNormalization())
    text_model.add(Dropout(0.25))

    text_model.add(Dense(units=1, activation="relu"))  # Adjust units for specific classes

    # Image (CNN) Model with ResNet blocks
    inputs = Input(shape=input_shape_1)
    
    # Initial Convolution and Pooling
    x = Conv2D(10, 5, padding='same', activation="relu")(inputs)
    x = MaxPooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    
    # Additional Convolution and Pooling Layers
    x = Conv2D(11, 7, padding='same', activation="relu")(x)
    x = MaxPooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    
    x = Conv2D(21, 3, padding='same', activation="relu")(x)
    x = MaxPooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    
    x = Conv2D(32, 3, padding='same', activation="relu")(x)
    x = MaxPooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    
    x = Conv2D(128, 3, padding='same', activation="relu")(x)
    x = MaxPooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    
    # Apply ResNet blocks
    x = resnet_block(x, filters=128)
    x = Conv2D(168, 3, padding='same', activation="relu")(x)
    x = MaxPooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    
    x = resnet_block(x, filters=168)
    
    # Flatten and fully connected layers
    x = Flatten()(x)
    x = Dense(338, activation="relu")(x)
    x = Dropout(0.15)(x)
    x = BatchNormalization()(x)
    
    x = Dense(338, activation="relu")(x)
    x = Dropout(0.1)(x)
    
    x = Dense(84, activation="relu")(x)
    x = Dropout(0.1)(x)
    
    # Concatenate Text and Image Model Outputs
    combined_output = concatenate([text_model.output, x])

    # Final Dense Layers for Classification
    x = Dense(16, activation="relu")(combined_output)
    x = Dense(num_classes, activation="softmax")(x)
    
    # Create the Model
    model = Model([text_model.input, inputs], x)
    
    return model

# Create and Compile the Model
mixed_data_model_resnet = create_mixed_nn_resnet()
mixed_data_model_resnet.compile(
    optimizer=Adam(learning_rate=0.0002),
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy', 'Precision', 'AUC']
)

# Print Model Summary
mixed_data_model_resnet.summary()

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import (
    Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout, concatenate, Add, Activation
)
from tensorflow.keras.optimizers import Adam

# Define the ResNet block
def resnet_block(x, filters, kernel_size=3, strides=1):
    """
    ResNet block with optional downsampling.
    
    Args:
    x (tensor): Input tensor.
    filters (int): Number of filters for the convolutional layers.
    kernel_size (int): Size of the convolutional kernel.
    strides (int): Stride size for the convolutional layers.

    Returns:
    tensor: Output tensor after applying the ResNet block operations.
    """
    shortcut = x
    x = Conv2D(filters, kernel_size, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    
    # Shortcut connection
    if strides != 1 or shortcut.shape[-1] != filters:
        shortcut = Conv2D(filters, 1, strides=strides, padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)
    
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

# Function to create your mixed CNN model with ResNet blocks
def create_mixed_nn_resnet(): 
    """
    Creates a mixed neural network model combining text (MLP) and image (CNN) inputs,
    including ResNet blocks for the image model.
    
    Returns:
    model (tf.keras.Model): Compiled Keras model.
    """
    input_shape_1 = (100, 200, 3)  # Input shape for image (CNN) model
    num_classes = 4
    
    # Text (MLP) Model
    text_model = Sequential()
    text_model.add(Dense(4, input_dim=3, activation="relu"))
    text_model.add(BatchNormalization())
    text_model.add(Dropout(0.25))
    
    text_model.add(Dense(6, activation="relu"))
    text_model.add(BatchNormalization())
    
    text_model.add(Dense(2, activation="relu"))
    text_model.add(BatchNormalization())
    text_model.add(Dropout(0.25))

    text_model.add(Dense(units=1, activation="relu"))  # Adjust units for specific classes

    # Image (CNN) Model with ResNet blocks
    inputs = Input(shape=input_shape_1)
    
    # Initial Convolution and Pooling
    x = Conv2D(16, 5, padding='same', activation="relu")(inputs)
    x = MaxPooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    
    # Additional Convolution and Pooling Layers
    x = Conv2D(32, 7, padding='same', activation="relu")(x)
    x = MaxPooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    
    x = Conv2D(64, 3, padding='same', activation="relu")(x)
    x = MaxPooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    
    x = Conv2D(96, 3, padding='same', activation="relu")(x)
    x = MaxPooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    
    x = Conv2D(192, 3, padding='same', activation="relu")(x)
    x = MaxPooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    
    # Apply ResNet blocks
    x = resnet_block(x, filters=192)
    x = Conv2D(256, 3, padding='same', activation="relu")(x)
    x = MaxPooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    
    x = resnet_block(x, filters=256)
    
    # Flatten and fully connected layers
    x = Flatten()(x)
    x = Dense(512, activation="relu")(x)
    x = Dropout(0.15)(x)
    x = BatchNormalization()(x)
    
    x = Dense(256, activation="relu")(x)
    x = Dropout(0.1)(x)
    
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.1)(x)
    
    # Concatenate Text and Image Model Outputs
    combined_output = concatenate([text_model.output, x])

    # Final Dense Layers for Classification
    x = Dense(8, activation="relu")(combined_output)
    x = Dense(num_classes, activation="softmax")(x)
    
    # Create the Model
    model = Model([text_model.input, inputs], x)
    
    return model

# Create the model
mixed_data_model_resnet = create_mixed_nn_resnet()

# Enable eager execution for debugging (optional)
tf.config.run_functions_eagerly(True)

# Compile the model
mixed_data_model_resnet.compile(
    optimizer=Adam(learning_rate=0.0002),
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy', 'Precision', 'AUC'],  # Add additional metrics
)

# Print model summary
mixed_data_model_resnet.summary()


In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import Callback

class CustomCheckpoint(Callback):
    """
    Custom Keras callback to save the model based on specified performance thresholds during training.

    Attributes:
        model (tf.keras.Model): The Keras model to be saved.
        save_path (str): The directory path where the model checkpoints will be saved.
        monitor_acc (str): The metric to monitor for validation accuracy (default is 'val_categorical_accuracy').
        monitor_prec (str): The metric to monitor for validation precision (default is 'val_precision').
        monitor_train_acc (str): The metric to monitor for training accuracy (default is 'categorical_accuracy').
        threshold (float): The threshold value for validation accuracy and precision above which the model will be saved.
        train_threshold (float): The threshold value for training accuracy above which the model will be saved.
    """

    def __init__(self, model, save_path, monitor_acc='val_categorical_accuracy', monitor_prec='val_precision', 
                 monitor_train_acc='categorical_accuracy', threshold=0.79, train_threshold=0.9):
        """
        Initializes the CustomCheckpoint callback.

        Args:
            model (tf.keras.Model): The Keras model to be saved.
            save_path (str): The directory path where the model checkpoints will be saved.
            monitor_acc (str, optional): The metric to monitor for validation accuracy (default is 'val_categorical_accuracy').
            monitor_prec (str, optional): The metric to monitor for validation precision (default is 'val_precision').
            monitor_train_acc (str, optional): The metric to monitor for training accuracy (default is 'categorical_accuracy').
            threshold (float, optional): The threshold value for validation accuracy and precision above which the model will be saved (default is 0.79).
            train_threshold (float, optional): The threshold value for training accuracy above which the model will be saved (default is 0.9).
        """
        super(CustomCheckpoint, self).__init__()
        self.model = model
        self.save_path = save_path
        self.monitor_acc = monitor_acc
        self.monitor_prec = monitor_prec
        self.monitor_train_acc = monitor_train_acc
        self.threshold = threshold
        self.train_threshold = train_threshold

    def on_epoch_end(self, epoch, logs=None):
        """
        Called at the end of each epoch during training to check and save the model based on specified thresholds.

        Args:
            epoch (int): The current epoch number.
            logs (dict): Dictionary containing the metrics from the current epoch.
        """
        val_acc = logs.get(self.monitor_acc)
        val_prec = logs.get(self.monitor_prec)
        train_acc = logs.get(self.monitor_train_acc)
        
        if val_acc is not None and val_prec is not None and train_acc is not None:
            # Check if validation accuracy, precision, and training accuracy meet the thresholds
            if val_acc > self.threshold and val_prec > self.threshold and train_acc > self.train_threshold:
                print(f'\nEpoch {epoch + 1}: val_categorical_accuracy, val_precision, and categorical_accuracy are above {self.threshold * 100}% - saving model to {self.save_path}')
                # Save the model with detailed metrics in the filename
                self.model.save(f"{self.save_path}_{val_acc:.4f}_{val_prec:.4f}_{train_acc:.4f}.h5")

# Sample usage
checkpoint_path = "alexnet_augmented"
custom_checkpoint = CustomCheckpoint(model=mixed_data_model, save_path=checkpoint_path)


In [ ]:
from datetime import datetime


def export_history_to_csv(history, filename):
    """
    Export the training history to a CSV file.

    Args:
        history (History): The History object returned from model.fit().
        filename (str): The filename to save the CSV file.

    Returns:
        None
    """
    # Convert the history.history dictionary to a DataFrame
    history_df = pd.DataFrame(history.history)
    
    # Save the DataFrame to a CSV file
    history_df.to_csv(filename, index=False)
    
    print(f"[INFO] Training history exported to {filename}")

print("[INFO] Starting model training...")
img_train = np.array(list(train_img))

# Get the current time before training starts
start_time = datetime.now()

# Print the starting time
print("[INFO] Training started at:", start_time)

# Train the model
history = mixed_data_model.fit(
    x=[train_attr[['age','sex','localization']], img_train], 
    y=np.array(train_y),
    validation_data=([test_attr[['age','sex','localization']], np.array(list(test_img))], np.array(test_y)),
    epochs=75, 
    batch_size=32,
    callbacks=[custom_checkpoint]
)

# Get the current time after training completes
end_time = datetime.now()

# Print the ending time
print("[INFO] Training ended at:", end_time)

# Example usage of export_history_to_csv
export_history_to_csv(history, 'ALEXNET_augmented_40min.csv')

In [ ]:
## Use to Save Model in case baseline metrics are not met


mixed_data_model_resnet.save("resnet_augmented_200_100__REDUCED_0.6_once_73.55_7791_8621.h5")

In [ ]:
import matplotlib.pyplot as plt

def plot_metrics(history, metric='auc', title='Metric', y_label='Metric Value', ylim=None):
    """
    Plot the specified metric from the training history.

    Args:
        history (History): The History object returned from model.fit().
        metric (str): The metric to plot from history (e.g., 'auc', 'precision', 'categorical_accuracy').
        title (str): The title of the plot.
        y_label (str): The label for the y-axis.
        ylim (tuple): Optional. Tuple specifying the y-axis limits (min, max).

    Returns:
        None
    """
    plt.figure(figsize=(12, 6))

    # Plot training metric
    plt.plot(history.history[metric], label=f'Training {metric.capitalize()}')

    # Plot validation metric
    val_metric = f'val_{metric}'
    plt.plot(history.history[val_metric], label=f'Validation {metric.capitalize()}')

    # Set title, labels, and fontsize
    plt.title(title, fontsize=16)
    plt.xlabel('Epoch', fontsize=16)
    plt.ylabel(y_label, fontsize=16)
    if ylim:
        plt.ylim(ylim)
    plt.legend(fontsize=16)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.show()

# Example usage, can be used to plot any metric in recorded history
plot_metrics(history, metric='auc', title='AUC', y_label='AUC', ylim=(0.1, 1.2))
plot_metrics(history, metric='precision', title='Precision', y_label='Precision', ylim=(0.1, 1.2))
plot_metrics(history, metric='categorical_accuracy', title='Categorical Accuracy', y_label='Categorical Accuracy', ylim=(0.1, 1.2))


### Testing Model


In [ ]:
## load testing dataset as per choice - original or reconstructed
dataset = pd.read_csv("E:\Final_Year_Project\Implementation\Image-Text-NN-SC-Detection\Testing\ISIC2018_Task3_Test_GroundTruth.csv")
dataset.shape


In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from glob import glob

# Mapping dictionary for disease labels
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma',
    'nc': "Non_Cancerous"
}

def preprocess_dataset(dataset, base_skin_dir):
    """
    Preprocesses the dataset to include human-readable labels, image paths, and resized images.

    Parameters:
    dataset (pd.DataFrame): The DataFrame containing the dataset.
    base_skin_dir (str): Base directory path where image files are stored.

    Returns:
    pd.DataFrame: Processed DataFrame with added columns for human-readable labels, image paths, and resized images.

    Raises:
    ValueError: If dataset is empty or base_skin_dir is invalid.
    """
    if dataset.empty:
        raise ValueError("Dataset is empty.")
    
    if not os.path.exists(base_skin_dir):
        raise ValueError(f"Directory '{base_skin_dir}' does not exist.")
    
    # Map dx codes to human-readable labels
    dataset['cell_type'] = dataset['dx'].map(lesion_type_dict.get) 
    dataset['disease_label'] = pd.Categorical(dataset['cell_type']).codes
    
    # Define image path dictionary
    imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x
                         for x in glob(os.path.join(base_skin_dir, '*', '*.jp*g'))}
    
    # Ensure 'image_id' is string type
    dataset['image_id'] = dataset['image_id'].astype(str)
    
    # Map image_id to image path
    dataset['path'] = dataset['image_id'].apply(lambda x: imageid_path_dict.get(x.replace('.png', ''), ''))
    
    # Drop rows with NaN values
    dataset.dropna(inplace=True)
    
    # Resize and load images into 'image' column
    dataset['image'] = dataset['path'].map(lambda x: np.asarray(Image.open(x).resize((200, 200))))
    
    return dataset

# Example usage
base_skin_dir = os.path.join('..', 'Data')  # Replace with actual base directory path
dataset = preprocess_dataset(dataset, base_skin_dir)
print(dataset.head())

# List of allowed values as in testing data too
allowed_values = ['nc', 'mel', 'nv']

# Update the dx column with allowed values
dataset['dx'] = dataset['dx'].apply(lambda x: x if x in allowed_values else 'others')
print(dataset['dx'].value_counts())


In [ ]:
## to sample 100 samples from each class
def sample_classes(df, column, n_samples=100, random_state=None):
    """
    Sample n_samples from each class of a specified column in a pandas DataFrame.

    Parameters:
    df (pd.DataFrame): The DataFrame to sample from.
    column (str): The name of the column to group by.
    n_samples (int): The number of samples to take from each class. Default is 100.
    random_state (int, optional): The random state for reproducibility. Default is None.

    Returns:
    pd.DataFrame: A DataFrame containing the sampled rows.
    """
    # Filter out rows where 'path' is an empty string
    df = df[df['path'] != ""]
    
    # Sample n_samples from each group in the specified column
    sampled_df = df.groupby(column).apply(lambda x: x.sample(n=n_samples, random_state=random_state))
    
    # Reset the index to get a clean DataFrame
    sampled_df = sampled_df.reset_index(drop=True)
    
    return sampled_df

# Example usage
# Assuming 'dataset' is your original DataFrame and 'dx' is the column to sample from
augmented_df = sample_classes(dataset, 'dx', n_samples=100)
print(f"Augmented Data Samples: {augmented_df.shape} \nDistribution: \n{augmented_df['dx'].value_counts()}")


In [ ]:
# Load the scaler object from the pickle file
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)
# Load the encoder object for localization from the pickle file
with open('encoder_localization.pkl', 'rb') as f:
    localization_encoder = pickle.load(f)
# Load the encoder object for sex from the pickle file
with open('encoder_sex.pkl', 'rb') as f:
    sex_encoder = pickle.load(f)

In [ ]:
# Preprocess the testing data
augmented_df['age'] = scaler.transform(augmented_df[['age']])
augmented_df['sex'] = sex_encoder.transform(augmented_df[['sex']])
augmented_df['localization'] = localization_encoder.transform(augmented_df[['localization']])

In [ ]:
# partition the 
# the data  90% for testing, 90 images form each class
print("[INFO] processing data...")
train_attr, test_attr, train_img, test_img = train_test_split(augmented_df[['age','sex', 'localization', 'cell_type_idx']], augmented_df['image'], test_size=0.9, random_state=4142)

## One hot encoding of variables
train_y = to_categorical(list(train_attr['cell_type_idx']),4)
test_y = to_categorical(list(test_attr['cell_type_idx']),4)


In [ ]:
import os
import pandas as pd
import h5py
import tensorflow as tf
from sklearn.metrics import classification_report, recall_score, precision_score, f1_score, accuracy_score

# Define input and output directories
modelname = "Custom_CNN"
input_dir = f'E:/Final_Year_Project/Models/{modelname}'
output_dir = f'E:/Final_Year_Project/Models/{modelname}/Analysis'

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Iterate through each file in the input directory
for filename in os.listdir(input_dir):
    # Check if the filename contains 'original_200_200' and ends with '.h5'
    if 'original_200_200' in filename and filename.endswith('.h5'):
        print(f"Evaluating Model: {filename}")
        model_path = os.path.join(input_dir, filename)
        
        # Load the model
        model = tf.keras.models.load_model(model_path)
        
        # Prepare test data (replace with your own data loading and preprocessing)
        y_pred = model.predict([test_attr[['age','sex','localization']], np.array(list(test_img))])
        y_pred = np.argmax(y_pred, axis=1)
        y_test = np.argmax(np.array(test_y), axis=1)
        
        # Calculate evaluation metrics
        recall = recall_score(y_test, y_pred, average='macro')
        precision = precision_score(y_test, y_pred, average='macro')
        f1_score_model = f1_score(y_test, y_pred, average='macro')
        accuracy_score1 = accuracy_score(y_test, y_pred)
        
        # Generate classification report
        report = classification_report(y_test, y_pred, output_dict=True)
        
        # Convert report to DataFrame
        df_report = pd.DataFrame(report).transpose()
        
        # Create DataFrame for overall metrics
        overall_metrics = pd.DataFrame({
            'Metric': ['Recall', 'Precision', 'F1_Score', 'Accuracy'],
            'Value': [recall, precision, f1_score_model, accuracy_score1]
        })
        
        # Combine overall metrics with classification report
        combined_df = pd.concat([overall_metrics, df_report])
        
        # Save evaluation results to CSV
        output_filename = f"{filename.replace('.h5', '')}_Testing.csv"
        combined_df.to_csv(os.path.join(output_dir, output_filename))
        ## This stores classification report in an output csv file for all models that are being eva;uated
        print(f"Evaluation results saved to: {os.path.join(output_dir, output_filename)}")


### Final Metrics

**Note:** Detailed evaluation metrics and results can be viewed in the "Results and Discussion" section of the project report. For comprehensive insights into model performance, please refer to the complete analysis provided in the report.

*Thank you for your interest and understanding.*

